In [1]:
import pyreadr
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import anndata

from dredFISH.Utils.__init__plots import *

# organize Johann et al

In [2]:
# annotation 
f = "./data/humanmethylation450_15017482_v1-2.csv"
annot = pd.read_csv(f, skiprows=7, dtype=str).set_index('IlmnID')
annot['CHR'] = annot['CHR'].fillna('NA')
annot['CHR'] = annot['CHR'].apply(lambda x: int(x) if not isinstance(x, str) else x) # fillna('NA')
annot['CHR'] = annot['CHR'].astype(str) 
print(annot['CHR'].unique())
annot

['Y' 'X' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' 'NA']


,Name,AddressA_ID,AlleleA_ProbeSeq,AddressB_ID,AlleleB_ProbeSeq,Infinium_Design_Type,Next_Base,Color_Channel,Forward_Sequence,Genome_Build,...,UCSC_RefGene_Group,UCSC_CpG_Islands_Name,Relation_to_UCSC_CpG_Island,Phantom,DMR,Enhancer,HMM_Island,Regulatory_Feature_Name,Regulatory_Feature_Group,DHS
IlmnID,,,,,,,,,,,,,,,,,,,,,
cg00035864,cg00035864,31729416,AAAACACTAACAATCTTATCCACATAAACCCTTAAATTTATCTCAA...,NaN,NaN,II,NaN,NaN,AATCCAAAGATGATGGAGGAGTGCCCGCTCATGATGTGAAGTACCT...,37,...,TSS1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg00050873,cg00050873,32735311,ACAAAAAAACAACACACAACTATAATAATTTTTAAAATAAATAAAC...,31717405,ACGAAAAAACAACGCACAACTATAATAATTTTTAAAATAAATAAAC...,I,A,Red,TATCTCTGTCTGGCGAGGAGGCAACGCACAACTGTGGTGGTTTTTG...,37,...,Body;TSS1500,chrY:9363680-9363943,N_Shore,NaN,NaN,NaN,Y:9973136-9976273,NaN,NaN,NaN
cg00061679,cg00061679,28780415,AAAACATTAAAAAACTAATTCACTACTATTTAATTACTTTATTTTC...,NaN,NaN,II,NaN,NaN,TCAACAAATGAGAGACATTGAAGAACTAATTCACTACTATTTGGTT...,37,...,Body;Body;Body,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg00063477,cg00063477,16712347,TATTCTTCCACACAAAATACTAAACRTATATTTACAAAAATACTTC...,NaN,NaN,II,NaN,NaN,CTCCTGTACTTGTTCATTAAATAATGATTCCTTGGATATACCAAGT...,37,...,Body,chrY:22737825-22738052,S_Shelf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg00121626,cg00121626,19779393,AAAACTAATAAAAATAACTTACAAACCAAATACTATACCCTACAAC...,NaN,NaN,II,NaN,NaN,AGGTGAATGAAGAGACTAATGGGAGTGGCTTGCAAGCCAGGTACTG...,37,...,Body,chrY:21664481-21665063,N_Shore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65622344,NORM_T,Purple,Norm_T59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57640346,NORM_C,Green,Norm_C60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47715450,NORM_T,Purple,Norm_T60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# metadata
f = './data/Johann_2016/sample_metadata.csv'
meta = pd.read_csv(f, skiprows=2).set_index("Sample Name")

f = './data/Johann_2016/sample_name_map.csv'
nmap = pd.read_csv(f, header=None)
nmap.columns = ['gsm', 'sample']
meta = meta.join(nmap.set_index('sample'))
meta = meta[~meta['gsm'].isnull()]
meta

,Molecular Subgroup (Consensus),Subgrouping based on 450K methylation data,Subrouping based on Affymetrix gene expression data,Subgrouping based on WGBS methylation data,Subgrouping based on H3K27Ac enhancer correlation,WGS cohort,RNASequencing cohort,WGBS cohort,H3K27Ac ChIP Sequencing cohort,BRD4 ChIP Sequencing cohort,OTX2 ChIP Sequencing cohort,MITF ChIP Sequencing cohort,"Gender (F= FeMale, M=Male)",Age at diagnosis,Localization of primary tumor,Type of SMARCB1 mutation in MLPA/Sanger Sequencing,Type of SMARCB1 mutation in MLPA/Sanger Sequencing,Previosuly published (PMID number),gsm
Sample Name,,,,,,,,,,,,,,,,,,,
dkfz_ATRT_1,SHH,SHH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,2.00,infratentorial,NaN,NaN,NaN,GSM1755170
dkfz_ATRT_2,SHH,SHH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,2.00,infratentorial,NaN,NaN,NaN,GSM1755171
dkfz_ATRT_3,SHH,SHH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,infratentorial,delExon5,delExon5,NaN,GSM1755172
dkfz_ATRT_4,MYC,MYC,NaN,MYC,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,M,4.00,infratentorial,delExon1_Exon9,delTBX1_NIPSNAP1,NaN,GSM1755173
dkfz_ATRT_5,SHH,SHH,NaN,SHH,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,M,4.00,supratentorial,c.197C>A p.Ser66*,delTBX1_NIPSNAP1,NaN,GSM1755174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dkfz_ATRT_146,SHH,SHH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,NaN,supratentorial,NaN,NaN,NaN,GSM2051701
dkfz_ATRT_147,SHH,SHH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,NaN,supratentorial,NaN,NaN,NaN,GSM2051702
dkfz_ATRT_148,SHH,SHH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,0.90,infratentorial,SMARCA4,NaN,NaN,GSM2051673


In [4]:
# mc data matrix
f = "./data/Johann_2016/Johann_2016_betas.h5ad"
adata = anndata.read(f)
adata

AnnData object with n_obs × n_vars = 485512 × 150

In [5]:
## combine all in one

# add annotation to the obs
adata.obs = annot.loc[adata.obs.index]

# add metadata to the var
name_lookup = meta.reset_index().set_index('gsm')['Sample Name']
adata.var.index = [name_lookup.loc[samp.split('_')[0]] for samp in adata.var.index]
adata.var = meta.loc[adata.var.index]

adata

AnnData object with n_obs × n_vars = 485512 × 150
    obs: 'Name', 'AddressA_ID', 'AlleleA_ProbeSeq', 'AddressB_ID', 'AlleleB_ProbeSeq', 'Infinium_Design_Type', 'Next_Base', 'Color_Channel', 'Forward_Sequence', 'Genome_Build', 'CHR', 'MAPINFO', 'SourceSeq', 'Chromosome_36', 'Coordinate_36', 'Strand', 'Probe_SNPs', 'Probe_SNPs_10', 'Random_Loci', 'Methyl27_Loci', 'UCSC_RefGene_Name', 'UCSC_RefGene_Accession', 'UCSC_RefGene_Group', 'UCSC_CpG_Islands_Name', 'Relation_to_UCSC_CpG_Island', 'Phantom', 'DMR', 'Enhancer', 'HMM_Island', 'Regulatory_Feature_Name', 'Regulatory_Feature_Group', 'DHS'
    var: 'Molecular Subgroup (Consensus)', 'Subgrouping based on 450K methylation data', 'Subrouping based on Affymetrix gene expression data', 'Subgrouping based on WGBS methylation data', 'Subgrouping based on H3K27Ac enhancer correlation', 'WGS cohort', 'RNASequencing cohort', 'WGBS cohort', 'H3K27Ac ChIP Sequencing cohort', 'BRD4 ChIP Sequencing cohort', 'OTX2 ChIP Sequencing cohort', 'MITF ChIP Se

In [6]:
output = "./data/Johann_2016/Johann_2016_betas_annotated.h5ad"
adata.write(output)

# organize our data

In [7]:
f = './data/mammal320k_to_mm10.RDS'
annot_m = pyreadr.read_r(f)[None]
annot_m = annot_m.set_index('Probe_ID')
annot_m = annot_m.astype(str)
print(annot_m.shape)
annot_m = annot_m[~annot_m['CGid'].isnull()]
annot_m['CHR'] = annot_m['seqnames'].astype(str).apply(lambda x: x[3:] if x.startswith('chr') else x)
print(annot_m.shape)
annot_m

(330948, 77)
(330948, 78)


,CGid,mammalianArrayProbes,mammalianProbesToUse.rmCor0.7Plus.ResMouseLessThan0.25.Calibration0.8Plus,havingReplicates,mapUniqWanding,SYMBOL,GeneRegionID,seqnames,probeStart,probeEnd,...,PMDstatusDetailed,WCGW_inCommonPMDs,WCGW_GenomWide,rmCor,rmP,groupWithHighResidual,maxResidual,groupWithHighResidualMouse,maxResidualMouse,CHR
Probe_ID,,,,,,,,,,,,,,,,,,,,,
cg00101675_BC21,cg00101675,yes,nan,no,nan,Ppp2r5e,Ppp2r5e_fiveUTR,12,75515719,75515768,...,commonHMD,nan,nan,nan,nan,nan,nan,nan,nan,12
cg00116289_BC21,cg00116289,yes,yes,no,nan,Zfr,Zfr_Exon,15,12170802,12170851,...,commonHMD,nan,nan,0.9894616374013984,2.3667978265695095e-23,Human_Artificial,0.2987134070729932,Mouse_Artificial,0.09230722904515432,15
cg00211372_TC21,cg00211372,yes,yes,no,nan,Rbmx,Rbmx_threeUTR,X,57387384,57387433,...,nan,nan,nan,0.9871442589270856,3.0955427221759218e-22,Human_Artificial,-0.6427299394644107,Mouse_Artificial,0.15953514375494113,X
cg00531009_BC21,cg00531009,yes,yes,no,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,0.9978278514898652,3.005895100945863e-32,Human_Artificial,-0.5757953634769454,Mouse_Liver,0.029698869474856913,nan
cg00896209_TC21,cg00896209,yes,yes,no,nan,Clk2,Clk2_Exon,3,89169680,89169729,...,commonHMD,nan,nan,0.9833152547439974,8.980566045143384e-21,Rhesus macaque_Artificial,-0.2576989066375933,Mouse_Liver,0.114425443553316,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg18473521_TC22,cg18473521,no,nan,yes,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
cg01027805_TC22,cg01027805,no,nan,yes,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
cg16543027_TC21,cg16543027,no,nan,no,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [8]:
f = 'SamplesheetAgeP39final.csv'
meta_m = pd.read_csv(f)
print(meta_m.shape)
meta_m = meta_m.set_index('Basename')
cols_remove = meta_m.filter(regex='clock')
meta_m = meta_m.drop(cols_remove, axis=1).astype(str)
print(meta_m.shape)
meta_m

(12, 229)
(12, 23)


,OriginalOrderInBatch,ExternalSampleID,Age,ConfidenceInAgeEstimate,CanBeUsedForAgingStudies,SpeciesLatinName,SpeciesCommonName,Tissue,Female,Folder,...,LongNameProject,Sample_Well,Date,Well,GiovanniniNumber,DNA.Weight.mg,DNA.Concentration,AgeMonths,Sex,Genotype
Basename,,,,,,,,,,,,,,,,,,,,,
206535570066_R05C02,1,20147-9 #1,0.333333333,100,yes,Mus musculus,Mouse,Olfactory Bulb,0,P39.ET0143BrianNa,...,Mammal320K_ET0158_ET0057_ET0120_ET0143_Plate1,C11,11/25/2020,A1,20147-9 #1,27.4,0.274,4.0,male,P0-CreC;Smarcb1(flox/flox)
206535570066_R06C02,2,20207-9 #1,0.416666667,100,yes,Mus musculus,Mouse,Olfactory Bulb,1,P39.ET0143BrianNa,...,Mammal320K_ET0158_ET0057_ET0120_ET0143_Plate1,D11,11/25/2020,B1,20207-9 #1,9.7,0.097,5.0,female,P0-CreC;Smarcb1(flox/flox)
206535570010_R01C01,3,17016-9 #1,0.391666667,100,yes,Mus musculus,Mouse,Trigeminal nerve,1,P39.ET0143BrianNa,...,Mammal320K_ET0158_ET0057_ET0120_ET0143_Plate1,E11,11/30/2020,C1,17016-9 #1,9.7,0.097,4.7,female,P0-CreC;Smarcb1(flox/flox)
206535570010_R02C01,4,17100-9B #1,0.275,100,yes,Mus musculus,Mouse,Meninges,0,P39.ET0143BrianNa,...,Mammal320K_ET0158_ET0057_ET0120_ET0143_Plate1,F11,11/30/2020,D1,17100-9B #1,10.2,0.102,3.3,male,P0-CreC;Smarcb1(flox/flox)
206535570010_R03C01,5,18161-10B #1,0.308333333,100,yes,Mus musculus,Mouse,Trigeminal nerve,1,P39.ET0143BrianNa,...,Mammal320K_ET0158_ET0057_ET0120_ET0143_Plate1,G11,11/30/2020,E1,18161-10B #1,4.7,0.047,3.7,female,P0-CreC;Smarcb1(flox/flox)
206535570010_R04C01,6,20074-9B #1,0.3,100,yes,Mus musculus,Mouse,Retro-orbital,1,P39.ET0143BrianNa,...,Mammal320K_ET0158_ET0057_ET0120_ET0143_Plate1,H11,12/1/2020,F1,20074-9B #1,8.6,0.086,3.6,female,P0-CreC;Smarcb1(flox/flox)
206535570010_R05C01,7,20095-9 #1,0.416666667,100,yes,Mus musculus,Mouse,Optic nerve,0,P39.ET0143BrianNa,...,Mammal320K_ET0158_ET0057_ET0120_ET0143_Plate1,A12,12/1/2020,G1,20095-9 #1,0.5,0.005,5.0,male,P0-CreC;Smarcb1(flox/flox)
206535570010_R06C01,8,20137-9 #1,0.258333333,100,yes,Mus musculus,Mouse,Olfactory bulb,0,P39.ET0143BrianNa,...,Mammal320K_ET0158_ET0057_ET0120_ET0143_Plate1,B12,12/1/2020,H1,20137-9 #1,10.1,0.101,3.1,male,P0-CreC;Smarcb1(flox/flox)
206535570010_R01C02,9,20142-9 #1,0.366666667,100,yes,Mus musculus,Mouse,Nasal cavity,1,P39.ET0143BrianNa,...,Mammal320K_ET0158_ET0057_ET0120_ET0143_Plate1,C12,12/1/2020,A2,20142-9 #1,10.7,0.107,4.4,female,P0-CreC;Smarcb1(flox/flox)


In [9]:
f = "all_probes_sesame_normalized.RDS" 
data_m = pyreadr.read_r(f)[None]
data_m = data_m.T
data_m

,206535570066_R05C02,206535570066_R06C02,206535570010_R01C01,206535570010_R02C01,206535570010_R03C01,206535570010_R04C01,206535570010_R05C01,206535570010_R06C01,206535570010_R01C02,206535570010_R02C02,206535570010_R03C02,206535570010_R04C02
cg00000165_TC21,0.464574,0.483712,0.485698,0.512629,0.488281,0.490644,0.496566,0.491678,0.485013,0.442666,0.507296,0.465939
cg00001209_TC21,0.709015,0.647783,0.642106,0.558514,0.670084,0.668659,0.692513,0.789740,0.638012,0.240190,0.714195,0.696545
cg00001364_TC21,0.749347,0.914103,0.914649,0.516546,0.899112,0.916861,0.748892,0.912464,0.913994,0.920995,0.818239,0.859054
cg00001582_TC21,0.054638,0.054335,0.053501,0.081808,0.057557,0.062748,0.038457,0.051540,0.060898,0.036618,0.067346,0.061484
cg00002920_TC21,0.199709,0.117025,0.298558,0.272371,0.214564,0.189437,0.321178,0.354263,0.193437,0.308441,0.354261,0.384626
...,...,...,...,...,...,...,...,...,...,...,...,...
rs877309_BC21,0.473372,0.546911,0.511721,0.494019,0.515623,0.496569,0.520476,0.518697,0.567449,0.480997,0.516118,0.508063
rs9292570_BC11,0.495280,0.503069,0.504700,0.506527,0.522663,0.530290,0.512177,0.484462,0.496929,0.480304,0.445632,0.487827
rs9363764_BC21,0.515029,0.459686,0.519578,0.508826,0.486397,0.479303,0.455853,0.487021,0.467514,0.541826,0.511584,0.463032
rs951295_BC11,0.056931,0.044640,0.060683,0.060909,0.055960,0.059086,0.056436,0.049847,0.047304,0.047698,0.076089,0.071503


In [10]:
adata_m = anndata.AnnData(
    X=data_m.values, 
    obs=annot_m.loc[data_m.index], 
    var=meta_m.loc[data_m.columns.values],
    )
adata_m

/tmp/ipykernel_760/743705661.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata_m = anndata.AnnData(


AnnData object with n_obs × n_vars = 326732 × 12
    obs: 'CGid', 'mammalianArrayProbes', 'mammalianProbesToUse.rmCor0.7Plus.ResMouseLessThan0.25.Calibration0.8Plus', 'havingReplicates', 'mapUniqWanding', 'SYMBOL', 'GeneRegionID', 'seqnames', 'probeStart', 'probeEnd', 'CGstart', 'CGend', 'width', 'strand', 'seq', 'annotation', 'geneChr', 'geneStart', 'geneEnd', 'geneLength', 'geneStrand', 'transcriptId', 'distanceToTSS', 'flank_txIds', 'flank_geneIds', 'flank_gene_distances', 'genic', 'Intergenic', 'Promoter', 'fiveUTR', 'threeUTR', 'Exon', 'Intron', 'downstream', 'distal_intergenic', 'main_Categories', 'CpG_number_on_geneRegion', 'CpGs_number_on_gene', 'ENTREZID', 'ENSEMBL', 'conservationInHuman', 'CGIslandNess', 'CG', 'stackHMM', 'stackHMM.Rob', 'repeatElements', 'GENENAME', 'CpGislandseqnames', 'CpGislandstart', 'CpGislandend', 'CpGislandwidth', 'CpGislandstrand', 'CpGislandlen', 'CpGislandcount', 'CpGislandgc', 'CpGislandoe', 'CpGislandden', 'CpGislandpvalue', 'CpGisland', 'Brown_r

In [11]:
output = "all_probes_sesame_normalized_annotated.h5ad"
adata_m.write(output)